# Implimenting Public Key Infrastructure
Digital certificates and public key infrastructure (PKI) are critical services used to manage identification, authentication, and data confidentiality across most private and public networks. 

### Objectives
- Implement certificates and certificate authorities.
- Implement PKI management.

## Implimenting Certificates and Certificate Authorities

A digital certificate is a public assertion of identity, validated by a certificate authority (CA). 
- Protecting web server communications
- Signing messages

### Public and Private key usage
Public key cryptography solves the problem of distributing encryption keys and Authenticate a message that you send to others.
- When u want others to send u confidential messages
    - Give them ur public key
    - Only u can decrypt the message

- When u want to authenticate urself to others
    - Create a signature and sign it by encrypting with ur private key
    - Others decrypt with ur public key

The problem is with knowing whom u communicating with even with secure communications.

#### Public Key Infrastructure(PKI) 
    - Aims to prove that the owners of public are who they say they are.
    - Public keys should be attached to digital certificate guranteed by Cerificate Authority(CA)

#### Certificate Authorities
Entity responsible for issuing and guaranteeing certificates. Private CAs can be set up within an organization for internal communications. Most network operating systems, including Windows Server, have certificate services. 
- Provide a range of certificate services useful to the community of users serviced by the CA.
- Ensure the validity of certificates and the identity of those applying for them (registration).
- Establish trust in the CA by users and government and regulatory authorities and enterprises, such as financial institutions.
- Manage the servers (repositories) that store and administer the certificates.
- Perform key and certificate lifecycle management, notably revoking invalid certificates.

#### PKI rust Models
Shows how users and different CAs are able to trust one another.
- Single CA
    Certificates issued by one CA(all users trust it)
    - Problems:
        - Exposed
        - Easily compromised and the PKI collapses

- Hierarchical (Intermediate CA)
    - A single CA (called the root) issues certificates to several intermediate CAs. The intermediate CAs issue certificates to subjects (leaf or end entities).
    - Certificate chaining(Chain of trust) - Each leaf certificate can be traced back to the root CA along the certification path.
    Most orgs trust multiple CAs.
    
    Advantage:
    -  Different intermediate CAs can be set up with different certificate policies, enabling users to perceive clearly what a particular certificate is designed for.
    The root is still a single point of failure
    - If the root is damaged or compromised, the whole structure collapses. Solved by taking the root server offline letting the intermediate CAs.

    Problem:
    - Limited opportunity for cross-certification(trusting the CA of another organization)
    - Two organizations could agree to share root CA, but with more orgs it becomes difficult. 

- Online vs Offline CAs
    * Online CAs - Available to accept and process certificate signing requests, publish certificate revocation lists, and perform other certificate management tasks. 
    * Offline CAs - it is disconnected from any network and usually kept in a powered-down state. The root CA will need to be brought online to add or update intermediate CAs.(cause of the risk of compromise)

#### REGISTRATION AUTHORITIES AND Certificate Signing Request (CSR) 
- Registration is the process by which end users create an account with the CA and become authorized to request certificates. 
- Certificate Signing Request - done when a subject wants to obtain a certificate. 
- The CSR is a Base64 ASCII file containing the information that the subject wants to use in the certificate, including its public key. The CA reviews the certificate and checks that the information is valid
    * For a web server, this may simply mean verifying that the subject name and fully qualified domain name (FQDN) are identical, and verifying that the CSR was initiated by the person administratively responsible for the domain, as identified in the domain's WHOIS records. If the request is accepted, the CA signs the certificate and sends it to the subject.
- The registration function may be delegated by the CA to one or more registration authorities (RAs). These entities complete identity checking and submit CSRs on behalf of end users, but they do not actually sign or issue certificates.

#### Digital Certificates
A digital certificate is essentially a wrapper for a subject's public key. It contains information about the subject and the certificate's issuer or guarantor. It is digitally signed  to prove that it was issued to the subject by a particular CA.
- Digital certificate details(based on the X.509 standard approved by the International Telecommunications Union and standardized by the Internet Engineering Taskforce) [link](https://datatracker.ietf.org/doc/html/rfc5208)
![image](../images/pki/i1.png)
- Public Key Infrastructure X.509 (PKIX) working group manages the development of these standards. 
- RSA also created a set of standards, referred to as Public Key Cryptography Standards (PKCS), to promote the use of public key infrastructure.

#### Certificate Attributes
The X.509 standard defines the fields or attributes that must be present in the certificate.
- Serial Number - A number uniquely identifying the certificate within the domain of its CA.
- Signature Algorithm - The algorithm used by the CA to sign the certificate.
- Issuer - The name of the CA
- Valid from/to - Date and time during which the certificate is valid.
- Subject - The name of the certificate holder, expressed as a distinguished name (DN). Within this, the common name (CN) part should usually match either the fully qualified domain name (FQDN) of the server or a user email address.
- Public Key - Public key and algorithm used by the certificate holder.
- Extensions - V3 certificates can be defined with extended attributes, such as friendly subject or issuer names, contact email addresses, and intended key usage.
- Subject alternative name (SAN) - This extension field is the preferred mechanism to specify additional host names for a single certificate.

#### Subject Name Attributes
- Common Name(CN) was initially used to identify FQDN by which the server is accessed. 
- Subject Alernative Name(SAN) extension field is structured to represent different types of identifiers, including domain names. If a certificate is configured with a SAN, the browser should validate that, and ignore the CN value. 
    - The SAN field also allows a certificate to represent different subdomains, such as www.comptia.org and members.comptia.org.
- It is still safer to put the FQDN in the CN as well, because not all browsers and implementations stay up to date with the standards.
- Listing the specific subdomains is more secure, but if a new subdomain is added, a new certificate must be issued. A wildcard domain, such as *.comptia.org, means that the certificate issued to the parent domain will be accepted as valid for all subdomains (to a single level)

#### Types of Certificate
Certificate policies define the different uses of certificate types issued by the CA. These can be configured as standard certificate templates.
Set by configuring the Key Usage Attribute. The extended key usage field(Enhance Key Usage) is a complimentary fo defining usage.  EKU field is more flexible than the Key Usage field, but problems can occur when nonstandard or vendor-specific definitions are used.
Values include:
- Server Authentication
- Client Authentication
- Code Signing
- Email protection

Extension can be tagged critical meaning application must process the extension correctly otherwise it will be rejected. It should be rejected if the application cannot resolve the key usage value.

#### Web server Certificate types
A server certificate guarantees the identity of e-commerce sites or any sort of website to which users submit data that should be kept confidential. 
* * Can be easily compromised hence some measures
- Domain Validation(DV) 
    * Proving ownership of particular domain.
    * This may be proved by responding to an email to the authorized domain contact or by publishing a text record to the domain.
    * This process can be highly vulnerable to compromise. 

- Extended Validation(EV) 
    * subjecting to a process that requires more rigorous checks on the subject's legal identity and control over the domain or software being signed. 
    * EV standards are maintained by the CA/Browser forum [link](cabforum.org)
    * Cannot be issued for a wilcard domain
    * fields needed to be filled [image](../images/pki/ev.jpg)

#### Other Certificate types
- Machine/Computer Certificates
    *  It might be necessary to issue certificates to machines regardless of function
    *  Machines without valid domain-issued certificates could be prevented from accessing network resources. 
    * The SAN and often the CN attribute should be set to the FQDN of the machine (host name and local domain part).

- Email/User Certificates
    * It can be used to sign and encrypt email messages, typically using Secure Multipurpose Internet Message Extensions (S/MIME) or Pretty Good Privacy (PGP).
    * The user's email address must be entered as the SAN and CN.

- Code Signing Certificates
    * A code signing certificate is issued to a software publisher, following some sort of identity check and validation process by the CA
    * Publisher then signs it to guarantee the validity of a software(executable/dlls) application or browser plug-in.
    * Required by some scripting environments, powershell
    * The CN is set to an organization name, such as "CompTIA Development Services, LLC," rather than an FQDN.

- Root certificate
    * The root certificate is the one that identifies the CA itself. The root certificate is self-signed. 
    * Uses key size of at least 2048 bites, now opting for 4096 bits
    * The CN for a root certificate is set to the organization/CA name, rather than FQDN

- Self-signed Certificates
    * Any host can be used with self signed though not recognised by OS or browser but admin can override it.

## Impliment PKI Management

#### Certificate and Key Management
Key management refers to operational considerations for the various stages in a key's life cycle. 
- Key Generation - creating a secure key pair of the required strength, using the chosen cipher.
- Certificate generation - for identity of public key, a cert id needed and verification by a CA is crucial with appropriate usage.
- Storage - the user must take steps to store the private key securely, ensuring that unauthorized access and use is prevented. 
- Revocation - if a private key is compromised, the key pair can be revoked to prevent users from trusting the public key.
- Expiration and renewal - a certificate key pair that has not been revoked expires after a certain period.

#### Key Recovery and Escrow
* The root's private key is very crucial.Access to such critical encryption keys must be logged and audited and is typically subject to M-of-N control
    - N - no of admins
    - M  - m of n must be present for access
    - M > 1, N > M
    - For 4 admins 2 must be present
*  Escrow means that something is held independently. 
    - Giving keys to 3rd party for orgs with no capability to handle.

#### Certificate Expiration
They are used with a limited with a limited duration(shelf-life) as set by CA policy. Root certs go for up to 10 yrs while web server certs go for 1 yr. Easier to renewal with an existing cert reather than a new one. It is possible to use the existing key (referred to specifically as certificate renewal) or generate a new key (the certificate is rekeyed). 

#### Certificate Revocation Lists
A cert maybe revoked or suspended
- A revoked one is no longer valid and cannot be unrevoked
- A suspended one can be re-enabled
    -  A suspended key is given the code Certificate Hold. 

CAs must maintain a certificate revocation list (CRL) of all revoked and suspended certificates, which can be distributed throughout the hierarchy. 
There is a risk that the certificate might be revoked but still accepted by clients because an up-to-date CRL has not been published. 
CRL attributes:
- Publish period - the date and time on which the CRL is published. Most CAs are set up to publish the CRL automatically.
- Distribution point - the location(s) to which the CRL is published.
- Validity period -  the period during which the CRL is considered authoritative. This is usually a bit longer than the publish period
- Signature - The CRL is signed by the CA

#### Online Certificate Status Protocol Responders(OSCP responder)
Rather than a whole CRL it returns the status of the requested certificate. 
- It is resource itensive for the CA running the OSCP server
- Privacy issue where OSCP could monitor/record client-browser requests
OCSP stapling resolves these issues by having the SSL/TLS web server periodically obtain a time-stamped OCSP response from the CA. When a client submits an OCSP request, the web server returns the time-stamped response, rather than making the client contact the OCSP responder itself.

#### Cerificate Pinning
The chain of trust is easy to compromise and an adversary can substitute a malicious but trusted certificate into the chain (using some sort of proxy or Man-in-the-Middle attack), they could be able to snoop on the supposedly secure connection.
Pinning refers to several techniques to ensure that when a client inspects the certificate is inspecting. This might be achieved by embedding the certificate data in the application code, or by submitting one or more public keys to an HTTP browser via an HTTP header, which is referred to as HTTP Public Key Pinning (HPKP).
HPKP is vulnerable [link]( (developer.mozilla.org/en-US/docs/Web/HTTP/Public_Key_Pinning). )

#### Certificate Formats
- Encoding - Cryptographic data—both certificates and keys—are processed as binary using Distinguished Encoding Rules (DER). Binary format files are not commonly used, however.
- File Extensions - A three character file extension is a convention, not a standard, and unfortunately file extensions do not always map cleanly to the type of encoding used within a certificate file, or even to the contents of a certificate file. 
    * Both .DER and .PEM can be used as file extensions, although the latter is not recognized by Windows. .PEM is the most widely used extension for ASCII format files in Linux.
    * he .CRT and .CER extensions can also be used, but they are not well-standardized. Most of the confusion arises from the way Windows handles certificates. In Linux, .CRT is most likely to represent an ASCII certificate. In Windows, the most common extension is .CER, but this does not tell you whether the file format is binary or ASCII.


#### Contents
A certificate file can also contain more than just a single certificate
- The PKCS #12 format allows the export of the private key with the certificate. 
- The P7B format implements PKCS #7, which is a means of bundling multiple certificates in the same file. 

#### OpenSSL
- In windows - cert infras is istalled and managed as active directory services. There is a certutil tool cl management/ powershell
In linux CA are implimented using OPENSSL suite [link](openssl.org)
openssl commands
- Root CA
    - To configure an RSA key pair is required
        > openssl genrsa -aes256 -out cakey.pem 4096
        - -aes256 - encrypts and requires password to use it
        - 4096 sets the len=gth
        - output is data in PEM ASCII format by default (cakey)
    
    - Generating a self-signed root x.509 digital cert
        > openssl req -config openssl.cnf -key cakey.pem -new -x509 -days 7300 -sha256 -out cacert.pem

#### Certificate Signing Requests
- To configure a certificate on a host, create a certificate signing request (CSR) with a new key pair. This command is run on the web server:
    > openssl req -nodes -new -newkey rsa:2048 -out www.csr -keyout www.key
- Then complete the prompts to enter the subject information for the certificate, taking care to match the common name (CN) to the FQDN by which clients access the server. 
- This key is created without a password, which would have to be input at any restart of the web server application. 
- This CSR file must then be transmitted to the CA server. On the CA, run the following command to sign the CSR and output the X.509 certificate
> openssl ca -config openssl.cnf -extensions webserver -infiles www.csr -out www.pem
- The passphrase must be entered to confirm use of the cakey.pem private key. 
-  The -extensions argument selects an area of the configuration file for a particular certificate type. This sets the key usage attribute, plus any other extended attributes that are needed.
View the new certificate to check the details using the following two commands:
> openssl x509 -noout -text -in www.pem
> openssl verify -verbose -cafile cacert.pem www.pem
- Transmit the www.pem file to the web server and update the server configuration to use it and the www.key private key.

#### Key and Certificate Management
You might export a copy of the private key from this server to be held in escrow as a backup. For this usage, you must password-protect the key:
> openssl rsa -aes256 -in www.key -out www.key.bak

- You might need to convert the certificate format to make it compatible with an application server, such as Java. The following command takes a PEM-encoded certificate and outputs a DER binary-encoded certificate
> openssl x509 -outform der -in www.pem -out www.der
- Another use case is to export a key and certificate for use in Windows:
> openssl pkcs12 -export -inkey www.key -in www.pem -out www.pfx

#### Certificate Issues
Most common is client rejecting server certificate or authentication server rejecting clients certificate
- If with existing cert, check if it has been not been expired/revoked/suspended
- If new cert
    - Check that the key usage settings are appropriate for the application. VPn/Email have very specific key usage configs
    - Check that the subject name is correctly configured and that the client is using the correct address.- if client tries IP rather than FQDN and Ip not configured
- If troubleshooting a new certificate that is correctly configured
    - check that clients have been configured with the appropriate chain of trust.
    - You need to install root and intermediate CA certificates on the client before a leaf certificate can be trusted
    -  Be aware that some client applications might maintain a different certificate store to that of the OS. 
- In either case, verify that the time and date settings on the server and client are synchronized. Incorrect date/time settings are a common cause of certificate problems. 

* You must also audit certificate infrastructure to ensure that only valid certificates are being issued and trusted.
* Review logs of issued certificates periodically. Validate the permissions of users assigned to manage certificate services. 
* Check clients to ensure that only valid root CA certificates are trusted. 
* Make sure clients are checking for revoked or suspended certificates. 